In [1]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import pytz
from numba import njit

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import base_optimizer

import vectorbt as vbt
from vectorbt.generic.nb import nanmean_nb
from vectorbt.portfolio.nb import order_nb, sort_call_seq_nb
from vectorbt.portfolio.enums import SizeType, Direction

In [2]:
# Define params
num_tests = 4000

vbt.settings.array_wrapper['freq'] = 'days'
vbt.settings.returns['year_freq'] = '252 days'
vbt.settings.portfolio['seed'] = 42
vbt.settings.portfolio.stats['incl_unrealized'] = True

In [4]:
# Get data
data = pd.read_pickle("data/yfinance_mag7.pkl")
data.head()

Price           Close                                                          \
Ticker           AAPL       AMZN      GOOGL        META        MSFT      NVDA   
Date                                                                            
2020-03-13  67.457481  89.250000  60.424614  169.632019  152.028717  5.998899   
2020-03-16  58.779289  84.457497  53.394722  145.454376  129.621109  4.891977   
2020-03-17  61.363819  90.391998  55.637001  148.851395  140.293671  5.411811   
2020-03-18  59.861629  91.500000  54.299900  146.400772  134.387878  5.051887   
2020-03-19  59.402973  94.046501  55.319023  152.547287  136.598984  5.304707   

Price                 Dividends             ... Stock Splits            \
Ticker           TSLA      AAPL AMZN GOOGL  ...         MSFT NVDA TSLA   
Date                                        ...                          
2020-03-13  36.441334       0.0  0.0   0.0  ...          0.0  0.0  0.0   
2020-03-16  29.671333       0.0  0.0   0.0  ...          0.0  0.0  0.0   
2020-03-17  28.680000       0.0  0.0   0.0  ...          0.0  0.0  0.0   
2020-03-18  24.081333       0.0  0.0   0.0  ...          0.0  0.0  0.0   
2020-03-19  28.509333       0.0  0.0   0.0  ...          0.0  0.0  0.0   

Price          Volume                                                      \
Ticker           AAPL       AMZN     GOOGL      META      MSFT       NVDA   
Date                                                                        
2020-03-13  370732000  176194000  79400000  35028600  92727400  634836000   
2020-03-16  322423600  178346000  96520000  39120400  87905900  726972000   
2020-03-17  324056000  218342000  83194000  34255600  81059800  833632000   
2020-03-18  300233600  192904000  93044000  37553100  81593200  874268000   
2020-03-19  271857200  207998000  74064000  39862300  85922700  765512000   

Price                  
Ticker           TSLA  
Date                   
2020-03-13  339604500  
2020-03-16  307342500  
2020-03-17  359919000  
2020-03-18  356793000  
2020-03-19  452932500  

[5 rows x 49 columns]

In [5]:
price = data['Close']

In [6]:
# Plot normalized price series
(price / price.iloc[0]).vbt.plot()

FigureWidget({
    'data': [{'name': 'AAPL',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'b3ba8945-7390-4f4f-8bcf-3529d1753a8e',
              'x': array([datetime.datetime(2020, 3, 13, 0, 0),
                          datetime.datetime(2020, 3, 16, 0, 0),
                          datetime.datetime(2020, 3, 17, 0, 0), ...,
                          datetime.datetime(2025, 3, 10, 0, 0),
                          datetime.datetime(2025, 3, 11, 0, 0),
                          datetime.datetime(2025, 3, 12, 0, 0)], dtype=object),
              'y': array([1.        , 0.87135316, 0.90966662, ..., 3.37219818, 3.27376581,
                          3.21654457])},
             {'name': 'AMZN',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f8719379-9c43-4010-9927-9ccdc2186d50',
              'x': array([datetime.datetime(2020, 3, 13, 0, 0),
                          datetime.datetime(2020, 3, 16, 0, 0),
    

In [7]:
returns = price.pct_change()

In [8]:
print(returns.mean())

Ticker
AAPL     0.001110
AMZN     0.000887
GOOGL    0.001012
META     0.001426
MSFT     0.000901
NVDA     0.002937
TSLA     0.002375
dtype: float64


In [9]:
print(returns.std())

Ticker
AAPL     0.018932
AMZN     0.022294
GOOGL    0.020050
META     0.027861
MSFT     0.018102
NVDA     0.034057
TSLA     0.041214
dtype: float64


In [10]:
print(returns.corr())

Ticker      AAPL      AMZN     GOOGL      META      MSFT      NVDA      TSLA
Ticker                                                                      
AAPL    1.000000  0.574376  0.613734  0.534545  0.706976  0.565570  0.485426
AMZN    0.574376  1.000000  0.636740  0.604032  0.672373  0.564936  0.432323
GOOGL   0.613734  0.636740  1.000000  0.619516  0.718190  0.567763  0.403932
META    0.534545  0.604032  0.619516  1.000000  0.600567  0.512263  0.334092
MSFT    0.706976  0.672373  0.718190  0.600567  1.000000  0.665585  0.441731
NVDA    0.565570  0.564936  0.567763  0.512263  0.665585  1.000000  0.468092
TSLA    0.485426  0.432323  0.403932  0.334092  0.441731  0.468092  1.000000


## vectorbt: Random search

### One-time allocation

In [13]:
np.random.seed(42)

# Generate random weights, n times
weights = []
for i in range(num_tests):
    w = np.random.random_sample(price.shape[1])
    w = w / np.sum(w)
    weights.append(w)

print(len(weights))

4000


In [14]:
# Build column hierarchy such that one weight corresponds to one price series
_price = price.vbt.tile(num_tests, keys=pd.Index(np.arange(num_tests), name='symbol_group'))
_price = _price.vbt.stack_index(pd.Index(np.concatenate(weights), name='weights'))

print(_price.columns)

MultiIndex([( 0.12377385005440206,    0,  'AAPL'),
            (  0.3141814830632648,    0,  'AMZN'),
            ( 0.24190121120621194,    0, 'GOOGL'),
            (  0.1978379931229667,    0,  'META'),
            ( 0.05155930389311296,    0,  'MSFT'),
            (0.051551332948848096,    0,  'NVDA'),
            ( 0.01919482571119334,    0,  'TSLA'),
            ( 0.20571128856353235,    1,  'AAPL'),
            (  0.1427609663966766,    1,  'AMZN'),
            ( 0.16816270349330772,    1, 'GOOGL'),
            ...
            ( 0.08282642690142582, 3998,  'MSFT'),
            ( 0.15307491390873518, 3998,  'NVDA'),
            (0.055463531981353544, 3998,  'TSLA'),
            ( 0.14294256566529503, 3999,  'AAPL'),
            (  0.3180184518129585, 3999,  'AMZN'),
            ( 0.08009918498222704, 3999, 'GOOGL'),
            (  0.3004771676928391, 3999,  'META'),
            ( 0.10490290644006016, 3999,  'MSFT'),
            (0.012790506706152173, 3999,  'NVDA'),
            ( 0

In [15]:
# Define order size
size = np.full_like(_price, np.nan)
size[0, :] = np.concatenate(weights)  # allocate at first timestamp, do nothing afterwards

print(size.shape)

(1256, 28000)


In [16]:
# Run simulation
pf = vbt.Portfolio.from_orders(
    close=_price,
    size=size,
    size_type='targetpercent',
    group_by='symbol_group',
    cash_sharing=True
) # all weights sum to 1, no shorting, and 100% investment in risky assets

print(len(pf.orders))

28000


In [18]:
# Plot annualized return against volatility, color by sharpe ratio
annualized_return = pf.annualized_return()
annualized_return.index = pf.annualized_volatility()
annualized_return.vbt.scatterplot(
    trace_kwargs=dict(
        mode='markers', 
        marker=dict(
            color=pf.sharpe_ratio(),
            colorbar=dict(
                title='sharpe_ratio'
            ),
            size=5,
            opacity=0.7
        )
    ),
    xaxis_title='annualized_volatility',
    yaxis_title='annualized_return'
).show()

In [19]:
# Get index of the best group according to the target metric
best_symbol_group = pf.sharpe_ratio().idxmax()

print(best_symbol_group)

1431


In [20]:
# Print best weights
print(weights[best_symbol_group])

[0.13926554 0.05846486 0.15091403 0.10267928 0.01654068 0.52736647
 0.00476915]


In [21]:
# Compute default stats
print(pf.iloc[best_symbol_group].stats())

Start                         2020-03-13 00:00:00
End                           2025-03-12 00:00:00
Period                         1256 days 00:00:00
Start Value                                 100.0
End Value                             1161.956549
Total Return [%]                      1061.956549
Benchmark Return [%]                   478.364032
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        57.587105
Max Drawdown Duration           373 days 00:00:00
Total Trades                                    7
Total Closed Trades                             0
Total Open Trades                               7
Open Trade PnL                        1061.956549
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


### Rebalance monthly

In [22]:
# Select the first index of each month
rb_mask = ~_price.index.to_period('m').duplicated()

print(rb_mask.sum())

61


In [23]:
rb_size = np.full_like(_price, np.nan)
rb_size[rb_mask, :] = np.concatenate(weights)  # allocate at mask

print(rb_size.shape)

(1256, 28000)


In [24]:
# Run simulation, with rebalancing monthly
rb_pf = vbt.Portfolio.from_orders(
    close=_price,
    size=rb_size,
    size_type='targetpercent',
    group_by='symbol_group',
    cash_sharing=True,
    call_seq='auto'  # important: sell before buy
)

print(len(rb_pf.orders))

1707993


In [25]:
rb_best_symbol_group = rb_pf.sharpe_ratio().idxmax()

print(rb_best_symbol_group)

100


In [26]:
print(weights[rb_best_symbol_group])

[0.21907155 0.02131672 0.13845641 0.05528918 0.02606821 0.4072032
 0.13259474]


In [27]:
print(rb_pf.iloc[rb_best_symbol_group].stats())

Start                                 2020-03-13 00:00:00
End                                   2025-03-12 00:00:00
Period                                 1256 days 00:00:00
Start Value                                         100.0
End Value                                       894.98427
Total Return [%]                                794.98427
Benchmark Return [%]                           478.364032
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                48.749417
Max Drawdown Duration                   378 days 00:00:00
Total Trades                                          190
Total Closed Trades                                   183
Total Open Trades                                       7
Open Trade PnL                                 363.626289
Win Rate [%]                                    92.349727
Best Trade [%]                                  569.41044
Worst Trade [%

In [31]:
def plot_allocation(rb_pf):
    # Plot weights development of the portfolio
    rb_asset_value = rb_pf.asset_value(group_by=False)
    rb_value = rb_pf.value()
    rb_idxs = np.flatnonzero((rb_pf.asset_flow() != 0).any(axis=1))
    rb_dates = rb_pf.wrapper.index[rb_idxs]
    fig = (rb_asset_value.vbt / rb_value).vbt.plot(
        trace_names=price.columns,
        trace_kwargs=dict(
            stackgroup='one'
        )
    )
    for rb_date in rb_dates:
        fig.add_shape(
            dict(
                xref='x',
                yref='paper',
                x0=rb_date,
                x1=rb_date,
                y0=0,
                y1=1,
                line_color=fig.layout.template.layout.plot_bgcolor
            )
        )
    fig.show()

In [32]:
plot_allocation(rb_pf.iloc[rb_best_symbol_group])  # best group

### Search and rebalance every 30 days

Utilize low-level API to dynamically search for best Sharpe ratio and rebalance accordingly. Compared to previous method, we won't utilize stacking, but do search in a loop instead. We also will use days instead of months, as latter may contain a various number of trading days.

In [34]:
srb_sharpe = np.full(price.shape[0], np.nan)
ann_factor = returns.vbt.returns.ann_factor

@njit
def pre_sim_func_nb(c, every_nth):
    # Define rebalancing days
    c.segment_mask[:, :] = False
    c.segment_mask[every_nth::every_nth, :] = True
    return ()

@njit
def find_weights_nb(c, price, num_tests):
    # Find optimal weights based on best Sharpe ratio
    returns = (price[1:] - price[:-1]) / price[:-1]
    returns = returns[1:, :]  # cannot compute np.cov with NaN
    mean = nanmean_nb(returns)
    cov = np.cov(returns, rowvar=False)  # masked arrays not supported by Numba (yet)
    best_sharpe_ratio = -np.inf
    weights = np.full(c.group_len, np.nan, dtype=np.float64)
    
    for i in range(num_tests):
        # Generate weights
        w = np.random.random_sample(c.group_len)
        w = w / np.sum(w)
        
        # Compute annualized mean, covariance, and Sharpe ratio
        p_return = np.sum(mean * w) * ann_factor
        p_std = np.sqrt(np.dot(w.T, np.dot(cov, w))) * np.sqrt(ann_factor)
        sharpe_ratio = p_return / p_std
        if sharpe_ratio > best_sharpe_ratio:
            best_sharpe_ratio = sharpe_ratio
            weights = w
            
    return best_sharpe_ratio, weights

@njit
def pre_segment_func_nb(c, find_weights_nb, history_len, ann_factor, num_tests, srb_sharpe):
    if history_len == -1:
        # Look back at the entire time period
        close = c.close[:c.i, c.from_col:c.to_col]
    else:
        # Look back at a fixed time period
        if c.i - history_len <= 0:
            return (np.full(c.group_len, np.nan),)  # insufficient data
        close = c.close[c.i - history_len:c.i, c.from_col:c.to_col]

    # Find optimal weights
    best_sharpe_ratio, weights = find_weights_nb(c, close, num_tests)
    srb_sharpe[c.i] = best_sharpe_ratio

    # Update valuation price and reorder orders
    size_type = SizeType.TargetPercent
    direction = Direction.LongOnly
    order_value_out = np.empty(c.group_len, dtype=np.float64)
    for k in range(c.group_len):
        col = c.from_col + k
        c.last_val_price[col] = c.close[c.i, col]
    sort_call_seq_nb(c, weights, size_type, direction, order_value_out)

    return (weights,)

@njit
def order_func_nb(c, weights):
    col_i = c.call_seq_now[c.call_idx]
    return order_nb(
        weights[col_i], 
        c.close[c.i, c.col],
        size_type=SizeType.TargetPercent
    )

In [35]:
# Run simulation using a custom order function
srb_pf = vbt.Portfolio.from_order_func(
    price,
    order_func_nb,
    pre_sim_func_nb=pre_sim_func_nb,
    pre_sim_args=(30,),
    pre_segment_func_nb=pre_segment_func_nb,
    pre_segment_args=(find_weights_nb, -1, ann_factor, num_tests, srb_sharpe),
    cash_sharing=True, 
    group_by=True
)


In [36]:
# Plot best Sharpe ratio at each rebalancing day
pd.Series(srb_sharpe, index=price.index).vbt.scatterplot(trace_kwargs=dict(mode='markers')).show()

In [37]:
print(srb_pf.stats())

Start                                 2020-03-13 00:00:00
End                                   2025-03-12 00:00:00
Period                                 1256 days 00:00:00
Start Value                                         100.0
End Value                                      466.504145
Total Return [%]                               366.504145
Benchmark Return [%]                           478.364032
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                59.205976
Max Drawdown Duration                   541 days 00:00:00
Total Trades                                          150
Total Closed Trades                                   143
Total Open Trades                                       7
Open Trade PnL                                  83.343912
Win Rate [%]                                    72.727273
Best Trade [%]                                 212.765829
Worst Trade [%

In [38]:
plot_allocation(srb_pf)

You can see how weights stabilize themselves with growing data.

In [39]:
# Run simulation, but now consider only the last 252 days of data
srb252_sharpe = np.full(price.shape[0], np.nan)

srb252_pf = vbt.Portfolio.from_order_func(
    price,
    order_func_nb,
    pre_sim_func_nb=pre_sim_func_nb,
    pre_sim_args=(30,),
    pre_segment_func_nb=pre_segment_func_nb,
    pre_segment_args=(find_weights_nb, 252, ann_factor, num_tests, srb252_sharpe),
    cash_sharing=True, 
    group_by=True
)

In [40]:
pd.Series(srb252_sharpe, index=price.index).vbt.scatterplot(trace_kwargs=dict(mode='markers')).show()

In [41]:
print(srb252_pf.stats())

Start                                 2020-03-13 00:00:00
End                                   2025-03-12 00:00:00
Period                                 1256 days 00:00:00
Start Value                                         100.0
End Value                                      221.596611
Total Return [%]                               121.596611
Benchmark Return [%]                           478.364032
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                53.213181
Max Drawdown Duration                   501 days 00:00:00
Total Trades                                          120
Total Closed Trades                                   113
Total Open Trades                                       7
Open Trade PnL                                 -10.453732
Win Rate [%]                                    57.522124
Best Trade [%]                                 156.919026
Worst Trade [%

In [42]:
plot_allocation(srb252_pf)

A much more volatile weight distribution.

## PyPortfolioOpt + vectorbt

### One-time allocation

In [43]:
# Calculate expected returns and sample covariance amtrix
avg_returns = expected_returns.mean_historical_return(price)
cov_mat = risk_models.sample_cov(price)

# Get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.max_sharpe()
clean_weights = ef.clean_weights()
pyopt_weights = np.array([clean_weights[symbol] for symbol in price.columns])

print(pyopt_weights)

[0.      0.      0.      0.      0.      0.99856 0.00144]


In [44]:
pyopt_size = np.full_like(price, np.nan)
pyopt_size[0, :] = pyopt_weights  # allocate at first timestamp, do nothing afterwards

print(pyopt_size.shape)

(1256, 7)


In [45]:
# Run simulation with weights from PyPortfolioOpt
pyopt_pf = vbt.Portfolio.from_orders(
    close=price,
    size=pyopt_size,
    size_type='targetpercent',
    group_by=True,
    cash_sharing=True
)

print(len(pyopt_pf.orders))

2


Faster than stacking solution, but doesn't let you compare weights.

In [46]:
print(pyopt_pf.stats())

Start                         2020-03-13 00:00:00
End                           2025-03-12 00:00:00
Period                         1256 days 00:00:00
Start Value                                 100.0
End Value                             1927.556231
Total Return [%]                      1827.556231
Benchmark Return [%]                   478.364032
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        66.280086
Max Drawdown Duration           373 days 00:00:00
Total Trades                                    2
Total Closed Trades                             0
Total Open Trades                               2
Open Trade PnL                        1827.556231
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


### Search and rebalance monthly

You can't use third-party optimization packages within Numba (yet).

Here you have two choices:
1) Use `os.environ['NUMBA_DISABLE_JIT'] = '1'` before all imports to disable Numba completely
2) Disable Numba for the function, but also for every other function in the stack that calls it

We will demonstrate the second option.

In [49]:
symbols = price.columns
def pyopt_find_weights(sc, price, num_tests):  # no @njit decorator = it's a pure Python function
    # Calculate expected returns and sample covariance matrix
    price = pd.DataFrame(price, columns=symbols)
    avg_returns = expected_returns.mean_historical_return(price)
    cov_mat = risk_models.sample_cov(price)

    # Get weights maximizing the Sharpe ratio
    ef = EfficientFrontier(avg_returns, cov_mat)
    weights = ef.max_sharpe()
    clean_weights = ef.clean_weights()
    weights = np.array([clean_weights[symbol] for symbol in symbols])
    best_sharpe_ratio = base_optimizer.portfolio_performance(weights, avg_returns, cov_mat)[2]
            
    return best_sharpe_ratio, weights

In [50]:
pyopt_srb_sharpe = np.full(price.shape[0], np.nan)

# Run simulation with a custom order function
pyopt_srb_pf = vbt.Portfolio.from_order_func(
    price,
    order_func_nb,
    pre_sim_func_nb=pre_sim_func_nb,
    pre_sim_args=(30,),
    pre_segment_func_nb=pre_segment_func_nb.py_func,  # run pre_segment_func_nb as pure Python function
    pre_segment_args=(pyopt_find_weights, -1, ann_factor, num_tests, pyopt_srb_sharpe),
    cash_sharing=True, 
    group_by=True,
    use_numba=False  # run simulate_nb as pure Python function
)

In [51]:
pd.Series(pyopt_srb_sharpe, index=price.index).vbt.scatterplot(trace_kwargs=dict(mode='markers')).show()

In [52]:
print(pyopt_srb_pf.stats())

Start                         2020-03-13 00:00:00
End                           2025-03-12 00:00:00
Period                         1256 days 00:00:00
Start Value                                 100.0
End Value                              783.920413
Total Return [%]                       683.920413
Benchmark Return [%]                   478.364032
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        67.338143
Max Drawdown Duration           563 days 00:00:00
Total Trades                                   41
Total Closed Trades                            39
Total Open Trades                               2
Open Trade PnL                         434.894282
Win Rate [%]                            64.102564
Best Trade [%]                         292.024223
Worst Trade [%]                        -40.717873
Avg Winning Trade [%]                   68.987261
Avg Losing Trade [%]                   -11.756039


In [53]:
plot_allocation(pyopt_srb_pf)